In [2]:
# importing modules

from bs4 import BeautifulSoup
import requests
import re
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import csv
import lxml

In [3]:

CSV = 'xxxxxx.csv'   # the name of the file in which the data will be imported
HOST = "https://xxxxxxx"  # the site from which you will receive data
URL = "https://xxxxxxx" # directory path on the site from where to import data



"""
# your browser data so that the server does not accept you as a bot, 
you can take this data in the browser settings as a developer
"""

HEADERS = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "user-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:86.0) Gecko/20100101 Firefox/86.0"} 




def get_html(url, params=''):
    
    """
    request function
    """
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    r = session.get(url, headers=HEADERS, params=params)
    
    return r




def get_data(html):
    
    """
    functions for parsing data from the catalog, 
    you can find this data in the browser menu for the developer
    """
    soup = BeautifulSoup(html, 'lxml')
    elements = soup.find_all('div', class_= 'sku-card-small-container') # selecting the main data class
   
    lenta = [] 
    
    for element in elements:
       
        try:
            lenta.append({
    
            'title': str(element.find('div', class_= 'sku-card-small__title').get_text(strip = True)), # selecting element data class
            'price': str(element.find('div', class_= 'sku-price--regular').get_text(strip = True)).replace("₽", "").replace(" ", ""),
            'price_discount': str(element.find('div', class_= 'sku-price--primary').get_text(strip = True)).replace("₽", "").replace(" ", "")
            })
          
        except TypeError:
                 print ("Error")

    return lenta




def save_csv(elements, path):
   
    """
    write to file function
    """
    
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['product', 'price', 'price_discount'])
        for element in elements:
             writer.writerow([element['title'], element['price'], element['price_discount']])
                
                
                
                
def parser():
    
    """ 
    functions parsing 
    """
    
    PAGENATION = input('Specify the number of pages ')
    PAGENATION = int(PAGENATION.strip())
    html = get_html(URL)
    
    if html.status_code == 200:
        lentas = []
        for page in range(1, PAGENATION+1):  # here you can set the cycle for which pages to parse
            print(f'Data parsing process: {page}')
            html = get_html(URL, params={'page': page})
            lentas.extend(get_data(html.text))
            save_csv(lentas, CSV)
        print('The parsing process is finished')    
        pass
    
    else:
        print('Error parsing')
        
       

In [ ]:
parser() # starting parsing